In [1]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_region_name

In [2]:
tf_version = '1.15.0'

In [3]:
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/ mnist --recursive

download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_labels.npy to mnist/eval_labels.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/train_labels.npy to mnist/train_labels.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_data.npy to mnist/eval_data.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/train_data.npy to mnist/train_data.npy


In [4]:
training_data_uri = "s3://sagemaker-sample-data-{}/tensorflow/mnist".format(region)

In [9]:
# mnist 훈련 스크립트
training_script = "../mnist.py"

In [10]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    # MNIST 훈련 script
    entry_point=training_script,
    role=role,
    instance_count=1,
    instance_type="ml.p2.xlarge",
    framework_version=tf_version,
    py_version="py3",
)

In [11]:
# Sagemaker 훈련 작업 시작
estimator.fit(training_data_uri)

2021-11-17 12:14:23 Starting - Starting the training job...
2021-11-17 12:14:47 Starting - Launching requested ML instancesProfilerReport-1637151263: InProgress
......
2021-11-17 12:15:47 Starting - Preparing the instances for training.........
2021-11-17 12:17:14 Downloading - Downloading input data...
2021-11-17 12:17:47 Training - Downloading the training image.....WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1467: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
2021-11-17 12:18:31,472 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2021-11-17 12:18:32,047 sagemaker-containers INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    

In [12]:
import numpy as np

!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/eval_data.npy eval_data.npy
!aws --region {region} s3 cp s3://sagemaker-sample-data-{region}/tensorflow/mnist/eval_labels.npy eval_labels.npy

eval_data = np.load("eval_data.npy")
eval_labels = np.load("eval_labels.npy")

download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_data.npy to ./eval_data.npy
download: s3://sagemaker-sample-data-us-west-2/tensorflow/mnist/eval_labels.npy to ./eval_labels.npy


In [13]:
# output_path 컴파일된 모델이 S3에 저장되는 위치
output_path = "/".join(estimator.output_path.split("/")[:-1])

# Neo compile할 대상 하드웨어 및 프레임워크 선택
optimized_estimator = estimator.compile_model(
    target_instance_family="ml_inf1",
    input_shape={"data": [1, 784]},  # Batch size 1, 1 channel, 28*28 image size.
    output_path=output_path
)

?????????????????????????????????.........................................................!

In [15]:
# 모델 최적화 후 실시간으로 예측 요청을 제공하는 Endpoint에 배포
optimized_predictor = optimized_estimator.deploy(
    initial_instance_count=1, instance_type="ml.inf1.xlarge"
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Using already existing model: tensorflow-training-2021-11-17-12-26-08-712


-----------!

In [17]:
import time

start = time.time()

for i in range(0, 10000):
    data = eval_data[i]
    predictions = optimized_predictor.predict(data)
print("time :", time.time() - start)

time : 67.73944997787476


In [18]:
optimized_predictor.delete_endpoint()